In [1]:
import re
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
columns = [u'cm_key', u'mvar2', u'mvar3', u'mvar4', u'mvar5', u'mvar6', u'mvar7', u'mvar8', u'mvar9', u'mvar10',
           u'mvar11', u'mvar12', u'mvar13', u'mvar14', u'mvar15', u'mvar16', u'mvar17', u'mvar18', u'mvar19', u'mvar20',
           u'mvar21', u'mvar22', u'mvar23', u'mvar24', u'mvar25', u'mvar26', u'mvar27', u'mvar28', u'mvar29', u'mvar30',
           u'mvar31', u'mvar32', u'mvar33', u'mvar34', u'mvar35', u'mvar36', u'mvar37', u'mvar38', u'mvar39', u'mvar40',
           u'mvar41', u'mvar42', u'mvar43', u'mvar44', u'mvar45', u'income_by_spending', u'membership_by_n',
           u'avg_electronics', u'avg_travel', u'avg_household', u'avg_car', u'avg_retailer', u'months_by_card',
           u'avg_quater1', u'avg_quater2', u'avg_quater3', u'avg_quater4', 
           # u'accepted_by_extended_supp',u'accepted_by_extended_elite', u'accepted_by_extended_credit', 
           u'avg_total', u'avg_sectors_quaters',
           u'other_spend_quater1_by_total', u'other_spend_quater2_by_total', u'other_spend_quater3_by_total',
           u'other_spend_quater4_by_total', u'other_spend_by_total', u'membership_by_income',u'spending_by_income',
           u'income_by_family', u'spending_by_family', u'membership_by_family'

]
quantity_features = [
                    u'mvar3', u'mvar6', u'mvar9', u'mvar16', u'mvar17', u'mvar18', u'mvar19', u'mvar20',
                    u'mvar21', u'mvar22', u'mvar23', u'mvar24', u'mvar25', u'mvar26', u'mvar27', u'mvar28', u'mvar29', 
                    u'mvar30', u'mvar31', u'mvar32', u'mvar33', u'mvar34', u'mvar35', u'mvar36', u'mvar37', u'mvar38', 
                    u'mvar39', u'avg_electronics', u'avg_travel', u'avg_household', u'avg_car', u'avg_retailer',
                    u'avg_quater1', u'avg_quater2', u'avg_quater3', u'avg_quater4', u'avg_total', u'avg_sectors_quaters',
                    u'income_by_family', u'spending_by_family', u'membership_by_family'
]
ratio_features = [
                u'mvar7', u'mvar8', u'mvar11',
                u'other_spend_quater1_by_total', u'other_spend_quater2_by_total', u'other_spend_quater3_by_total',
                u'other_spend_quater4_by_total', u'other_spend_by_total', u'months_by_card', 
                # u'accepted_by_extended_supp', u'accepted_by_extended_elite', u'accepted_by_extended_credit', 
                u'income_by_spending', u'membership_by_n',u'membership_by_income',u'spending_by_income'
]
number_features = [
                 u'mvar13', u'mvar14', u'mvar15', u'mvar2', u'mvar4', u'mvar5', u'mvar40',
                 u'mvar41', u'mvar42', u'mvar43', u'mvar44', u'mvar45'
]

categorical_columns = [u'mvar12', u'mvar10']
labels = pd.Series(train.target.values)
train = train[columns]
test = test[columns]

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for feature in categorical_columns:
    train[feature] = le.fit_transform(train[feature])
    test[feature] = le.fit_transform(test[feature])
ltr = len(train)
test_key = test.cm_key
data = pd.concat([train, test], axis = 0)
data.drop(['cm_key'], axis = 1, inplace = True)
del train
del test


# Transform the skewed numeric features by taking log(feature + 1).
# from scipy.stats import skew
# skewed = data[quantity_features].apply(lambda x: skew(x.dropna().astype(float)))
# skewed = skewed[skewed > 0.75]
# skewed = skewed.index
# data[skewed] = np.log1p(data[skewed])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(data[quantity_features])
scaled = scaler.transform(data[quantity_features])
for i, col in enumerate(quantity_features):
    data[col] = scaled[:,i]

data['mvar32'].fillna(0,inplace=True)
data['mvar33'].fillna(0,inplace=True)
data['mvar34'].fillna(0,inplace=True)
data['mvar35'].fillna(0,inplace=True)
data['avg_retailer'].fillna(0,inplace=True)
data['avg_quater1'].fillna(0,inplace=True)
data['avg_quater2'].fillna(0,inplace=True)
data['avg_quater3'].fillna(0,inplace=True)
data['avg_quater4'].fillna(0,inplace=True)
data['avg_sectors_quaters'].fillna(0,inplace=True)
train_data_df = data[:ltr]
test_data_df = data[ltr:]

In [7]:
def printtofile(df, PROBABILITY_THRESHOLD):
    df.drop('0',1, inplace=True)
    filtr = lambda x: 1 if x > PROBABILITY_THRESHOLD else 0
    df['Credit'] = df['1'].apply(filtr)
    df['Elite'] = df['2'].apply(filtr)
    df['Supp'] = df['3'].apply(filtr)
    print df[['Credit', 'Elite', 'Supp']].values.sum()
    df['prob'] = df['1']*df['Credit'] + df['2']*df['Elite'] + df['3']*df['Supp']
    df['cm_key'] = range(50001, 50001+len(df))
    df.sort_values(['prob'], ascending=[False], inplace=True)
    df.drop(['1', '2', '3'],1, inplace=True)
    df['label_type'] = 6*df['Credit'] + 8*df['Elite'] + 10*df['Supp']
    lbl = lambda x: 'Credit' if x == 6 else ('Elite' if x == 8 else ('Supp' if x == 10 else 'Credit'))
    df['label'] = df['label_type'].apply(lbl)
    df.to_csv("results_keras.csv", index=False)

In [8]:
print train_data_df.shape
# train_data_df.astype(float)

(40000, 68)


In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
print train_data_df.info()

# standarise data before training
scaler = StandardScaler().fit(train_data_df)
train_data_df = scaler.transform(train_data_df)
test_data_df = scaler.transform(test_data_df)
print train_data_df.shape
print test_data_df.shape
print labels.shape


Using TensorFlow backend.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 39999
Data columns (total 68 columns):
mvar2                           40000 non-null int64
mvar3                           40000 non-null float64
mvar4                           40000 non-null int64
mvar5                           40000 non-null int64
mvar6                           40000 non-null float64
mvar7                           40000 non-null float64
mvar8                           40000 non-null float64
mvar9                           40000 non-null float64
mvar10                          40000 non-null int64
mvar11                          40000 non-null float64
mvar12                          40000 non-null int64
mvar13                          40000 non-null int64
mvar14                          40000 non-null int64
mvar15                          40000 non-null int64
mvar16                          40000 non-null float64
mvar17                          40000 non-null float64
mvar18                     

In [10]:
from keras import optimizers
def keras_model(train):
    input_dim = train.shape[1]
    classes = 4
    
    model = Sequential()
    model.add(Dense(100, activation = 'tanh', input_shape = (input_dim,))) #layer 1
    model.add(Dense(30, activation = 'tanh')) #layer 2
    model.add(Dense(classes, activation = 'sigmoid')) #output
    opt = optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer = opt, 
                  loss='categorical_crossentropy',
                  metrics = ['accuracy'])
    return model
callback = EarlyStopping(monitor='val_acc',patience=3)

In [11]:
Y_train = to_categorical(labels)

In [12]:
model = keras_model(train_data_df)
model.fit(train_data_df, Y_train, batch_size=30, epochs=85, callbacks=[callback], shuffle=True)

Epoch 1/85
40000/40000 [==============================] - 2s - loss: 0.7186 - acc: 0.7729     
Epoch 2/85
 3030/40000 [=>............................] - ETA: 1s - loss: 0.6855 - acc: 0.7825

/home/innovationchef/.local/lib/python2.7/site-packages/keras/callbacks.py:496: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: acc,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


40000/40000 [==============================] - 1s - loss: 0.6924 - acc: 0.7784     
Epoch 3/85
40000/40000 [==============================] - 1s - loss: 0.6860 - acc: 0.7783     
Epoch 4/85
40000/40000 [==============================] - 1s - loss: 0.6824 - acc: 0.7786     
Epoch 5/85
40000/40000 [==============================] - 1s - loss: 0.6793 - acc: 0.7784     
Epoch 6/85
40000/40000 [==============================] - 1s - loss: 0.6750 - acc: 0.7782     
Epoch 7/85
40000/40000 [==============================] - 1s - loss: 0.6724 - acc: 0.7782     
Epoch 8/85
40000/40000 [==============================] - 1s - loss: 0.6702 - acc: 0.7781     
Epoch 9/85
40000/40000 [==============================] - 1s - loss: 0.6675 - acc: 0.7783     
Epoch 10/85
40000/40000 [==============================] - 1s - loss: 0.6621 - acc: 0.7787     
Epoch 11/85
40000/40000 [==============================] - 1s - loss: 0.6609 - acc: 0.7789     
Epoch 12/85
40000/40000 [==============================] - 

In [13]:
# check validation accuracy
vpreds = model.predict_proba(test_data_df)
submission = pd.DataFrame(vpreds, columns=['0','1','2','3'])
submission.to_csv("raw_results.csv")
printtofile(submission,0.275)

 8608/10000 [========================>.....] - ETA: 0s4138
